<a href="https://colab.research.google.com/github/woranov/amlta-project/blob/main/notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
try:
    import amlta  # noqa F401
except ImportError:
    %pip install --no-dependencies git+https://github.com/woranov/amlta-project.git

In [2]:
from pathlib import Path

from tqdm.auto import tqdm

try:
    from google.colab import drive  # pyright: ignore[reportMissingImports]

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from amlta.config import config
from amlta.formatting.data import create_process_section
from amlta.formatting.yaml import format_as_yaml
from amlta.probas import processes

In [3]:
if IN_COLAB:
    mount_point = Path("/content/drive")
    drive_path = mount_point / "MyDrive"

    # edit
    data_dir = drive_path / "uni" / "ws2425" / "amlta" / "project" / "data"

    config.update(data_dir=data_dir)

    if not mount_point.exists():
        drive.mount(str(mount_point))

In [4]:
# by default lci result only
process_uuids = processes.read_uuids()

In [5]:
def process_to_yaml(process: processes.ProcessData):
    return format_as_yaml(create_process_section(process, include_flows=False))


def save_yamls(uuids: list[str]):
    directory = config.ilcd_dir / "processes_yaml_2"
    directory.mkdir(exist_ok=True)

    process_iter = processes.ProcessData.from_uuids(uuids)
    for process in tqdm(process_iter, total=len(uuids)):
        yaml_str = process_to_yaml(process)
        file = directory / f"{process.processInformation.dataSetInformation.UUID}.yaml"
        if file.exists():
            continue
        with open(file, "w", encoding="utf-8") as f:
            f.write(yaml_str)

In [6]:
save_yamls(process_uuids)

  0%|          | 0/11762 [00:00<?, ?it/s]